<a href="https://colab.research.google.com/github/Siertx/Aprendizaje-automatico/blob/main/Pr%C3%A1ctica_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import urllib.request
import zipfile
import random
import numpy as np
from os import remove
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold
import pandas as pd

In [ ]:
url = 'https://archive.ics.uci.edu/static/public/320/student+performance.zip'
urllib.request.urlretrieve(url, 'student+performance.zip')

extract_dir = 'sample_data/'
with zipfile.ZipFile('student+performance.zip', 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
    remove('student+performance.zip') #<- al terminar borramos el ZIP

extract_dir = 'sample_data/'
with zipfile.ZipFile(extract_dir+'student.zip', 'r') as zip_ref:
      zip_ref.extractall(extract_dir)
      remove(extract_dir+'student.zip') #<- al terminar borramos el ZIP

df1 = pd.read_csv(extract_dir+'student-por.csv', sep=';')
df2 = pd.read_csv(extract_dir+'student-mat.csv', sep=';')

df = pd.merge(df1, df2, on=["school", "sex", "age", "address", "famsize", "Pstatus", "Medu", "Fedu", "Mjob", "Fjob", "reason", "nursery", "internet"])

In [ ]:
flag_save_test = False
fraction_test = 0.2

idx = df.index.tolist()
random.shuffle(idx)

N,D = df.shape

N_test = int(N*fraction_test)
testset = df.iloc[idx[:N_test]]
trainset = df.iloc[idx[N_test:]]

print(f'El tamaño del conj. de entrenamiento es: ',trainset.shape)
print(f'El tamaño del conj. de test es: ',testset.shape)

if flag_save_test:
  test_folder = "....." #<- ruta de la carpeta, por ejemplo "sample_data/"
  test_name   = "....." #<- nombre del CSV donde guardamos testSet
  testset.to_csv(test_folder + test_name, dec='.', sep=';', header=True)

El tamaño del conj. de entrenamiento es:  (306, 53)
El tamaño del conj. de test es:  (76, 53)


In [ ]:
print(trainset.head(10))

    school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
45      GP   F   15       U     LE3       A     4     3     other     other   
319     GP   M   18       U     GT3       T     4     4     other     other   
254     GP   M   16       U     GT3       T     3     3   at_home     other   
250     GP   M   22       U     GT3       T     3     1  services  services   
50      GP   F   16       U     LE3       T     2     2  services  services   
368     MS   F   18       U     GT3       T     3     3     other     other   
133     GP   F   15       U     GT3       T     1     1   at_home     other   
116     GP   M   15       R     GT3       T     2     1    health  services   
64      GP   F   15       U     LE3       T     4     3  services  services   
344     MS   M   17       U     GT3       T     3     3    health     other   

     ... famrel_y freetime_y  goout_y  Dalc_y  Walc_y health_y absences_y  \
45   ...        5          2        2       1       1

Separamos los datos:

In [ ]:
col_num = trainset.select_dtypes(include=['int64', 'float64']).columns
col_cat = trainset.select_dtypes(include=['object', 'category']).columns


Escalamos


In [ ]:
scalar = MinMaxScaler()
trainset[col_num] = scalar.fit_transform(trainset[col_num])

Filtramos aquellos que tengan menos de 0.03 de varianza

In [ ]:
selector = VarianceThreshold(threshold=0.03)
selector.set_output(transform="pandas")
selector.fit(trainset[col_num])
df_SemiFiltrao = selector.transform(trainset[col_num])

Filtramos ahora por correlación entre columnas con un coeficiente superior a 0.95

Averiguamos la correlación entre ellas

In [ ]:
correlacion = df_SemiFiltrao.corr()

Apuntamos al triángulo superior de la matriz para evitar duplicados y la diagonal

In [ ]:
upper = correlacion.where(np.triu(np.ones(correlacion.shape), k=1).astype(bool))

Identificamos las columnas con un coeficiente superior a 0.95

In [ ]:
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

Sacamos las columnas anteriormente detectadas

In [ ]:
df_filtrao = df.drop(columns=to_drop)
print(df_filtrao.head(10))

  school sex       age address famsize Pstatus  Medu  Fedu      Mjob  \
0     GP   F  0.428571       U     GT3       A  1.00  1.00   at_home   
1     GP   F  0.285714       U     GT3       T  0.25  0.25   at_home   
2     GP   F  0.000000       U     LE3       T  0.25  0.25   at_home   
3     GP   F  0.000000       U     GT3       T  1.00  0.50    health   
4     GP   F  0.142857       U     GT3       T  0.75  0.75     other   
5     GP   M  0.142857       U     LE3       T  1.00  0.75  services   
6     GP   M  0.142857       U     LE3       T  0.50  0.50     other   
7     GP   F  0.285714       U     GT3       A  1.00  1.00     other   
8     GP   M  0.000000       U     LE3       A  0.75  0.50  services   
9     GP   M  0.000000       U     GT3       T  0.75  1.00     other   

       Fjob  ... schoolsup_y famsup_y  paid_y  activities_y  higher_y  \
0   teacher  ...         yes       no      no            no       yes   
1     other  ...          no      yes      no            no  

Codificamos los atributos categoricos

In [ ]:
le = LabelEncoder()
for col in col_cat:
    df_filtrao[col] = le.fit_transform(df_filtrao[col])

In [ ]:
print(df_filtrao.head(10))

   school  sex       age  address  famsize  Pstatus  Medu  Fedu  Mjob  Fjob  \
0       0    0  0.428571        1        0        0  1.00  1.00     0     4   
1       0    0  0.285714        1        0        1  0.25  0.25     0     2   
2       0    0  0.000000        1        1        1  0.25  0.25     0     2   
3       0    0  0.000000        1        0        1  1.00  0.50     1     3   
4       0    0  0.142857        1        0        1  0.75  0.75     2     2   
5       0    1  0.142857        1        1        1  1.00  0.75     3     2   
6       0    1  0.142857        1        1        1  0.50  0.50     2     2   
7       0    0  0.285714        1        0        0  1.00  1.00     2     4   
8       0    1  0.000000        1        1        0  0.75  0.50     3     2   
9       0    1  0.000000        1        0        1  0.75  1.00     2     2   

   ...  schoolsup_y  famsup_y  paid_y  activities_y  higher_y  romantic_y  \
0  ...            1         0       0             0  